In [1]:
import numpy as np
import matplotlib.pyplot as plt


# Function to calculate density
def calc_density(f_ijk):
    return np.einsum('ijk->jk', f_ijk)


# Function to calculate the velocity field at each grid point
def calc_vel_field(f_ijk, c_ij, rho_ij):
    return np.einsum('ji, ikl -> jkl', c_ij.T, f_ijk)/rho_ij

# Streaming function
def streaming(f_ijk, c_ij):
    for i in range(1,9):
        f_ijk[i] = np.roll(f_ijk[i,:,:], shift=c_ij[i], axis=(0,1))
    return f_ijk

# The Bounce-Back function
def bounce_back(f_ijk, couette_b=False, poiseuille_b=False, u_wall=[], rho_wall=[], w_i=[], c_ij=[]):
    if couette_b == True:
        '''
        Only 2 boundaries: a fixed bottom boundary and a moving top boundary
        We follow periodic boundary conditions for the left and right boundaries
        Normal bounce-back for the bottom wall
        '''
        f_ijk[2, :, 1] = f_ijk[4, :, 0]
        f_ijk[5, :, 1] = np.roll(f_ijk[7, :, 0], 1)
        f_ijk[6, :, 1] = np.roll(f_ijk[8, :, 0], -1)
        '''
        Bounce back on the top moving wall is done differently
        The velocity is taken into consideration to determine the new position
        The normal bounce back is applied to the channel perpendicular to the moving wall
        '''
        # the density near the wall is just the sum 
        f_ijk[4, :, -2] = f_ijk[2, :, -1]
        f_ijk[7, :, -2] = np.roll(f_ijk[5, :, -1], -1) - 6. * w_i[5] * rho_wall * u_wall[0]
        f_ijk[8, :, -2] = np.roll(f_ijk[6, :, -1], 1) + 6. * w_i[6] * rho_wall * u_wall[0]

    elif poiseuille_b == True:
        '''
        We have 2 fixed boundaries on parallel sides. So regular bounce back on both sides.
        '''
        f_ijk[2, :, 1] = f_ijk[4, :, 0]
        f_ijk[5, :, 1] = np.roll(f_ijk[7, :, 0], 1)
        f_ijk[6, :, 1] = np.roll(f_ijk[8, :, 0], -1)
        f_ijk[4, :, -2] = f_ijk[2, :, -1]
        f_ijk[7, :, -2] = np.roll(f_ijk[5, :, -1], -1)
        f_ijk[8, :, -2] = np.roll(f_ijk[6, :, -1], 1)
#         f_ijk[[4,7,8], :, -2] = f_ijk[[2,5,6], :, -1]
    else:
        '''
        Channels 1,2,3 and 4
        '''
        f_ijk[1, 1, :] = f_ijk[3, 0, :]
        f_ijk[2, :, 1] = f_ijk[4, :, 0]
        f_ijk[3, -2, :] = f_ijk[1, -1, :]
        f_ijk[4, :, -2] = f_ijk[2, :, -1]
        
        '''
        Channels 5 and 6
        '''
        # Channel 5 <-> Channel 7
        f_ijk[5, :, 1] = np.roll(f_ijk[7, :, 0], 1)
        f_ijk[5, 1, :] = np.roll(f_ijk[7, 0, :], 1)
        f_ijk[6, :, 1] = np.roll(f_ijk[8, :, 0], -1)
        f_ijk[6, -2, :] = np.roll(f_ijk[8, -1, :], 1)

        '''
        Moving top boundary
        '''
        f_ijk[7, -2, :] = np.roll(f_ijk[5, -1, :], -1)
        f_ijk[7, :, -2] = np.roll(f_ijk[5, :, -1], -1) - 6. * w_i[5] * rho_wall * u_wall[0]
        f_ijk[8, 1, :] = np.roll(f_ijk[6, 0, :], -1)
        f_ijk[8, :, -2] = np.roll(f_ijk[6, :, -1], 1) + 6. * w_i[6] * rho_wall * u_wall[0]
        
    return f_ijk


def PBC(f_ijk, w_i, c_ij, pin, pout, u_ijk):
    '''
    Periodicity is done in such a way that we imagine 
    '''
    fneq1_ij = f_ijk[:,1,:]
    fneqN_ij = f_ijk[:,-2,:]
    rho1_i = np.einsum('ij -> j', fneq1_ij)
    rhoN_i = np.einsum('ij -> j', fneqN_ij)
    u1_ij = np.einsum('ij, jk -> ik', c_ij.T, fneq1_ij)/rho1_i
    uN_ij = np.einsum('ij, jk -> ik', c_ij.T, fneqN_ij)/rhoN_i
    u1sq_i = np.einsum('ij -> j', u1_ij * u1_ij)
    uNsq_i = np.einsum('ij -> j', uN_ij * uN_ij)
    feqpin_ij = np.ones_like(fneq1_ij)
    feqpout_ij = np.ones_like(fneqN_ij)
    feq1_ij = np.ones_like(fneq1_ij)
    feqN_ij = np.ones_like(fneqN_ij)
    # Loop through all the channels
    for i in range(0,9):
        cu1_i = np.einsum('n, nj -> j', c_ij[i], u1_ij)
        cuN_i = np.einsum('n, nj -> j', c_ij[i], uN_ij)
        cu1sq_ij = np.einsum('i -> i', cu1_i * cu1_i)
        cuNsq_ij = np.einsum('i -> i', cuN_i * cuN_i)
        feqpin_ij[i] = w_i[i] * pin * (1 + 3 * cuN_i + (9/2) * cuNsq_ij - (3/2) * uNsq_i)
        feqpout_ij[i] = w_i[i] * pout * (1 + 3 * cu1_i + (9/2) * cu1sq_ij - (3/2) * u1sq_i)
        feq1_ij[i] = w_i[i] * rho1_i * (1 + 3 * cu1_i + (9/2) * cu1sq_ij - (3/2) * u1sq_i)
        feqN_ij[i] = w_i[i] * rhoN_i * (1 + 3 * cuN_i + (9/2) * cuNsq_ij - (3/2) * uNsq_i)
    f0_ij = feqpin_ij + (fneqN_ij - feqN_ij)
    fN1_ij = feqpout_ij + (fneq1_ij - feq1_ij)
    return f0_ij, fN1_ij
        

# Function to find the equilibrium distribution function
def calc_feq(w_i, c_ij, rho_ij, u_ijk, NY, NX):
    eq_f_ijk = np.ones([9,NY,NX])
    u2_ij = np.einsum('ijk -> jk', u_ijk * u_ijk)
    # Loop through all the channels
    for i in range(0,9):
        cu_ij = np.einsum('n, njk -> jk', c_ij[i], u_ijk)
        cu2_ij = np.einsum('ij -> ij', cu_ij * cu_ij)
        eq_f_ijk[i] = w_i[i] * rho_ij * (1 + 3 * cu_ij + (9/2) * cu2_ij - (3/2) * u2_ij)
    return eq_f_ijk


# Collision function
def collision(f_ijk, c_ij, w_i, omega):
    # Calculating the density and velocity
    rho_ij = np.einsum('ijk->jk', f_ijk)
    u_ijk = np.einsum('ji, ikl -> jkl', c_ij.T, f_ijk)/rho_ij
    NZ, NY, NX = f_ijk.shape
    eq_f_ijk = calc_feq(w_i, c_ij, rho_ij, u_ijk, NY, NX)
    f_ijk = f_ijk + omega * (eq_f_ijk - f_ijk)
    return f_ijk, rho_ij, u_ijk